# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp (°C),Humidity %,Cloudiness %,Wind Speed (m/s),Country,Date
0,0,ushuaia,-54.8000,-68.3000,5.06,100,100,1.40,AR,1731183096
1,1,utrik,11.2278,169.8474,28.48,80,76,9.82,MH,1731183330
2,2,vadso,70.0744,29.7487,2.09,87,75,4.12,NO,1731183332
3,3,bethel,41.3712,-73.4140,12.05,29,0,5.14,US,1731183333
4,4,malango,-2.9627,119.9001,18.26,96,80,1.32,ID,1731183334


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Longitude",  
    "Latitude", 
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity %", 
    scale=0.1,        # Adjust the scale as necessary
    color="City"   
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity %)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp (°C)'] > 20) &  # Example condition for temperature
    (city_data_df['Max Temp (°C)'] < 40) &  # Example condition for temperature
    (city_data_df['Humidity %'] < 98)       # Example condition for humidity
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Latitude,Longitude,Max Temp (°C),Humidity %,Cloudiness %,Wind Speed (m/s),Country,Date
1,1,utrik,11.2278,169.8474,28.48,80,76,9.82,MH,1731183330
6,6,adamstown,-25.0660,-130.1015,22.94,87,99,8.88,PN,1731183336
7,7,edd,13.9297,41.6925,30.69,52,0,9.78,ER,1731183337
9,9,avarua,-21.2078,-159.7750,23.03,60,100,3.60,CK,1731183340
10,10,alofi,-19.0595,-169.9187,25.94,65,20,7.72,NU,1731183341


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Latitude', 'Longitude', 'Humidity %']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Latitude,Longitude,Humidity %,Hotel Name
1,utrik,MH,11.2278,169.8474,80,
6,adamstown,PN,-25.0660,-130.1015,87,
7,edd,ER,13.9297,41.6925,52,
9,avarua,CK,-21.2078,-159.7750,60,
10,alofi,NU,-19.0595,-169.9187,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 100000  # Set the radius to 10,000 meters
params = {
    "apiKey": "geoapify_key",  # Replace with your actual Geoapify API key
    "limit": 1,                 # Limit to 1 hotel
    "type": "accommodation"      # Specify the type of places to search for
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Latitude']
    longitude = row['Longitude']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude}, {latitude},{radius}"  # Define the search area
    params["bias"] = f"proximity: {longitude}, {latitude}"           # Bias towards the location

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request
    response = requests.get(f"{base_url}appid={geoapify_key}")
                              
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # Handle unsuccessful requests
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
utrik - nearest hotel: API request failed
adamstown - nearest hotel: API request failed
edd - nearest hotel: API request failed
avarua - nearest hotel: API request failed
alofi - nearest hotel: API request failed
san patricio - nearest hotel: API request failed
saint-philippe - nearest hotel: API request failed
alotau - nearest hotel: API request failed
tumayr - nearest hotel: API request failed
gashua - nearest hotel: API request failed
kone - nearest hotel: API request failed
puerto ayora - nearest hotel: API request failed
port mathurin - nearest hotel: API request failed
la libertad - nearest hotel: API request failed
marsa alam - nearest hotel: API request failed
aden - nearest hotel: API request failed
enewetak - nearest hotel: API request failed
pokaran - nearest hotel: API request failed
georgetown - nearest hotel: API request failed
kota kinabalu - nearest hotel: API request failed
kulhudhuffushi - nearest hotel: API request failed
dapa - nearest hotel: A

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)